# This pipeline runs using python3.10.12

In [33]:
import sys
sys.version_info

sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)

import sys
print(sys.version)

In [34]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# Bash usage 

In [35]:
%%bash
echo "CUT and RUN analysis"

CUT and RUN analysis


## To calculate counts from bam files use deeptools package following the command lines bellow. Our sample bam file are located ate samples folder

In [37]:
%%bash
cd Examples/samples/; 
multiBamSummary BED-file --BED ../../hg38_CUTnRUN_greenlist.v1.bed --smartLabels -e --centerReads -o glist_quant.npz  -b GSM5956324.bam GSM5956325.bam GSM5956326.bam GSM5956336.bam GSM5956337.bam GSM5956338.bam --outRawCounts output
cat output | tr -d "'#" | sed $'s/\t/_/1' | cut -f 1,3- > glist_quant.tsv
echo "Counts were calculated!!"

Number of bins found: 869


Counts were calculated!!


## To calculate Normalization Factors using counts, follow the command line bellow. The file glist_quant.tsv used here is in counts folder once the glist_quant.tsv built in the previous step was a result of subsampled bam files, thus it doesn't represent the complete dataset. 

In [38]:
%%bash
Rscript --vanilla original_scripts/get_sizeFactors.R Examples/counts/glist_quant.tsv Examples/counts/glist_sizeFactors.tsv

Loading required package: DESeq2
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading required package: SummarizedExperiment
Loading requi

DESeq2 
  TRUE 


## Here, we only describe the peak calling of Pax3FOXO1. First, we have to build a file "ratio-size-factor.txt" (example is in peaks folder). The ratio parameter is calculated [[size factor from treatment / size factor from IgG]]. Examples to perform peak calling from 3 replicates are shown bellow. The samples and data were obtained from GSE19853 (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE198753). Just keep in mind that in this demo, the bam files are subsampled from a larger dataset.

In [39]:
%%bash 
c=`grep Rep1 Examples/peaks/ratio-size-factor.txt | cut -f 2`
macs2 callpeak -t Examples/samples/GSM5956336.bam -c Examples/samples/GSM5956324.bam -n Examples/peaks/Pax3FOXO1-rep1 --ratio ${c} --format BAMPE -g hs --keep-dup all

c=`grep Rep2 Examples/peaks/ratio-size-factor.txt | cut -f 2`
macs2 callpeak -t Examples/samples/GSM5956337.bam -c Examples/samples/GSM5956325.bam -n Examples/peaks/Pax3FOXO1-rep2 --ratio ${c} --format BAMPE -g hs --keep-dup all

c=`grep Rep3 Examples/peaks/ratio-size-factor.txt | cut -f 2`
macs2 callpeak -t Examples/samples/GSM5956338.bam -c Examples/samples/GSM5956326.bam -n Examples/peaks/Pax3FOXO1-rep3 --ratio ${c} --format BAMPE -g hs --keep-dup all

INFO  @ Fri, 15 Dec 2023 10:39:39: 
# Command line: callpeak -t Examples/samples/GSM5956336.bam -c Examples/samples/GSM5956324.bam -n Examples/peaks/Pax3FOXO1-rep1 --ratio 0.6944264 --format BAMPE -g hs --keep-dup all
# ARGUMENTS LIST:
# name = Examples/peaks/Pax3FOXO1-rep1
# format = BAMPE
# ChIP-seq file = ['Examples/samples/GSM5956336.bam']
# control file = ['Examples/samples/GSM5956324.bam']
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Using a custom scaling factor: 6.94e-01
# Range for calculating regional lambda is: 1000 bps and 10000 bps
# Broad region calling is off
# Paired-End mode is on
 
INFO  @ Fri, 15 Dec 2023 10:39:39: #1 read fragment files... 
INFO  @ Fri, 15 Dec 2023 10:39:39: #1 read treat

## To perform the statistical analysis, you have to merge the peaks found in the replicates and then measure counts for this new BED file.

In [40]:
%%bash
#merge peaks 
cat Examples/peaks/*summits.bed | bedtools sort -i - | bedtools merge -i - > Examples/peaks/Pax3FOXO1-peaks-merged.bed
# calculate counts
cd Examples/samples; 
multiBamSummary BED-file --BED ../peaks/Pax3FOXO1-peaks-merged.bed --smartLabels -e --centerReads -o ../peaks/glist_quant.npz  -b GSM5956324.bam GSM5956325.bam GSM5956326.bam GSM5956336.bam GSM5956337.bam GSM5956338.bam --outRawCounts ../peaks/output
cat ../peaks/output | tr -d "'#" | sed $'s/\t/_/1' | cut -f 1,3- > ../peaks/glist_quant.tsv
echo "Counts were calculated!!"

Number of bins found: 272


Counts were calculated!!


## To perfom the normalization and entropy calculations, we use here the peaks/example_glist_quant.tsv data for demontration, but you should use the peaks/glist_quant.tsv resulted from your analysis. 

In [41]:
%%bash
#calculate entropy
Rscript --vanilla original_scripts/norm+entropy.R Examples/peaks/example_glist_quant.tsv Examples/results/Pax3FOXO1

Loading required package: ggplot2
Loading required package: broman
Loading required package: entropy
Loading required package: reshape2


 ggplot2   broman  entropy reshape2 
    TRUE     TRUE     TRUE     TRUE 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  2.157   2.681   2.758   2.737   2.815   2.890 
null device 
          1 
